In [68]:
import cryptography


# PL2 : Symmetric Cryptography + Hash Functions & MACs

Exercício 1: We can use the python cryptography library (pyca/cryptography) to compute one block with AES 


In [97]:
from os import urandom
from binascii import hexlify
from cryptography .hazmat. primitives .ciphers import Cipher, algorithms, modes
key = urandom (16)
iv = urandom (16)
cipher = Cipher( algorithms .AES(key), modes.ECB ())
encryptor = cipher. encryptor ()
# What happens i f you don ' t p a s s 16−b y t e i n p u t ?
ct = encryptor.update(b"attack at dawn!!") + encryptor.finalize()
print(hexlify(key))
cphFile = open("ciphertext.bin", "wb")
cphFile.write(ct)
cphFile.close ()


b'2869125513ee10945cfc8fed4efa916f'


Exercise 2 - We can use openSSL in the console to invest the block

In [98]:
!openssl enc -aes-128-ecb -nopad -d -K 2869125513ee10945cfc8fed4efa916f -in ciphertext.bin 


attack at dawn!!


A inversão funcionou 


Study and understand the options used in this command(run openssl, and then help aes-128-ecb)

In [53]:
!openssl help aes-128-ecb

Usage: aes-128-ecb [options]

General options:
 -help               Display this summary
 -list               List ciphers
 -ciphers            Alias for -list
 -e                  Encrypt
 -d                  Decrypt
 -p                  Print the iv/key
 -P                  Print the iv/key and exit
 -engine val         Use engine, possibly a hardware device

Input options:
 -in infile          Input file
 -k val              Passphrase
 -kfile infile       Read passphrase from file

Output options:
 -out outfile        Output file
 -pass val           Passphrase source
 -v                  Verbose output
 -a                  Base64 encode/decode, depending on encryption flag
 -base64             Same as option -a
 -A                  Used with -[base64|a] to specify base64 buffer as a single line

Encryption options:
 -nopad              Disable standard block padding
 -salt               Use salt in the KDF (default)
 -nosalt             Do not use salt in the KDF
 -debug          

 

-aes-128-ecb -> signfica que é um advanced encryption standard, com 128 bits, em que o modo de bloco é Eletronic Codebook(ECB) 

-nopad -> significa que não há padding (é usados nos casos que estamos a criptogifar dados que já são um múltiplo exato do tamanho do bloco do algoritmo de criptografia,e portanto, não necessitam de "padding")

-d -> Significa decrypt 

-K val -> Key e o valor da chave  

-in infile -> por um ficheiro para aplicar a chave e o algoritmo 

3 - Modify the python example above to encrypt a file in CBC mode

In [72]:
from os import urandom
from binascii import hexlify
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
key = urandom(16) #Gerar um chave random de 16 bits
iv = urandom(16)
cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
encryptor = cipher.encryptor()
ct = encryptor.update(b"attack at dawn!!") + encryptor.finalize()
print(hexlify(key))
print(hexlify(iv))
cphFile = open("ciphertext.bin","wb")
cphFile.write(ct)
cphFile.close()


b'2dd9e1fe862c6e63114292f349face2c'
b'c681a315f2f02db70463a1808489d52b'


colocamos o modo CBC (Cipher Block Chaining) na linha 6 e coloca-mos como argumento um vetor inicializador gerado automaticamente com 16 bits pela biblioteca os (urandom). 
O resto permanece igual :) 

4 - Decrypt the file with OpenSSL and check for success.

Para decryptar usamos o comando em cima e mudamos o modo

In [75]:
!openssl enc -aes-128-cbc  -nopad -d -iv c681a315f2f02db70463a1808489d52b -K 2dd9e1fe862c6e63114292f349face2c -in ciphertext.bin 

/[��ݵk�
�Q�͡


Como podemos ver falta também o vetor inicializador, pelo que mudaremos o codigo para também dar print ao "iv". 
Mesmo assim, a frase vem com simbolo estranhos e nao como uma string como seria suposto? 

5 - Edit the file to change the value of(but not delete!) one byte and decrypt again. You can use ghex editor in Linux  or equivelent.

Já alteramos um byte, agora vamos ver se funciona. 

In [74]:
!openssl enc -aes-128-cbc -nopad -d -iv aa4e700d39ef2e1a3794c8cc0adfb996 -K c4f7ca11cfcc62e719ca2d05233333e2 -in ciphertext.bin 

����{�o	7ݑWl��


 - What happened? The text wasn't decrypted correted, produzing some sort of letters that aren't readable. 
 - Could you recover a file encrypted with CBC if the IV and the first ciphertext block were corrupted or lost? 

 The IV is crucial for decrypting the first block of ciphertext. Each subsequent block of ciphertext depends on the correct decryption of the previous block due to the chaining mechanism. 

Conclusion: The loss of IV and the first block of ciphertext in CBC mode severaly impacts the recoverability of the encrypted file, especially the first and partially the second block of plaintext, the rest of the file beyond the second block can often be recovery, albeit with potencial loss of context or integrity for the overall data. 

 - Could you recover it if during a satellite transmission one bit of the ciphertext is not delivered?

If during a satellite transmission one bit of the cipher text is not delivered, this will have direct impact of the block and more extensive corruption in the sibsequent block, the rest of the data remains unaffected and can be recovered. The feasibilitiy of correcting the error depends on the data's nature, the use of error correction techniques, and the error's context. 
 

 - Could you modify a byte in the middle of a CBC encrypted file without fully re-encrypting it?

Modify a byte in  a ciphertext block: 
Affected block: The decryption of this block will be corrupted at the location of the modification. Since each plaintext block is produced by XORing the decrypted ciphertext blocl with the previous ciphertext block, changina byte in the cipher text will result in a corresponding chanfe in the decrypted plaintext. The next block will also be affected, but in a more unpredictable manner. The chaining mechanism means that the entire next block of plaintext wull be corrupted, not just a byte.

Short Answer: no

6 - Repeat the exercise with CTR mode. What are the differences? 

In [103]:
from os import urandom
from binascii import hexlify
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
key = urandom(16) #Gerar um chave random de 16 bits
nonce = urandom(16)
cipher = Cipher(algorithms.AES(key), modes.CBC(nonce))
encryptor = cipher.encryptor()
ct = encryptor.update(b"attack at dawn!!") + encryptor.finalize()
print(hexlify(key))
print(hexlify(nonce))
cphFile = open("ciphertext_ctr.bin","wb")
cphFile.write(ct)
cphFile.close()

b'0e045e3e64883debfc1c21e292e07256'
b'8f81a232d56b4d421ffcd2fa0066fda9'


In [106]:
!openssl enc -aes-128-ctr -d -iv 8f81a232d56b4d421ffcd2fa0066fda9 -K 0e045e3e64883debfc1c21e292e07256 -in ciphertext_ctr.bin

x�4�ہ]��tֹ�n�


Ver se existem diferencas em alterar um byte?

In [107]:
!openssl enc -aes-128-ctr   -d -iv 8f81a232d56b4d421ffcd2fa0066fda9 -K 0e045e3e64883debfc1c21e292e07256 -in ciphertext_ctr.bin



x�4�ہ]��tֹ�n�


We can see that are not differences between the plaintext generateds before and after altering one byte, this happens because the CTR mode works in parallel, while the CBC mode 
works in chains. Altering one byte of the ciphertext doensn't imply the loss of massive information in the CTR mode as it means in CBC mode. 

7 - Use OpenSSL to compute the SHA-256 hash value of a file(it can be the pdf file for this Lab assign then write a Python program that recomputes the same SHA-256 for the same file)

To generate a SHA-256 hash using OpenSSL for a file
usamos a opcao dgst e -sha256 e depois o caminho para o ficheiro

In [110]:
!openssl dgst -sha256 "2-Symmetric Cryptography + Hash Functions + MACs.pdf"

SHA2-256(2-Symmetric Cryptography + Hash Functions + MACs.pdf)= f8b14260a51b0405f29ff8bb2c765a0fe92aa6f6b34f3e0c0ec56452a81cd39e


In [3]:
import hashlib

openedFile = open('2-Symmetric Cryptography + Hash Functions + MACs.pdf',"rb")
readFile = openedFile.read()
hash_SHA256 = hashlib.sha256(readFile).hexdigest()
print(hash_SHA256)


f8b14260a51b0405f29ff8bb2c765a0fe92aa6f6b34f3e0c0ec56452a81cd39e


8 - Use the tool available (https://alf.nu/SHA1) to construct two PDFS with the same SHA-1 value. One of the PDFS should explain how a single SHA-1 collision allows finding infinite pairs of colliding PDFs